In [1]:
import numpy as np 
import matplotlib.pyplot as plt

import pyOpenLPT as lpt
redirector = lpt.PythonStreamRedirector()

def print_dict(dict):
    for key in dict:
        print(key+":", dict[key])

### StereoMatch class

In [2]:
# load cam 
cam = []
intensity_max = []
useid_list = []
for i in range(4):
    file = '../../test/inputs/test_StereoMatch/cam' + str(i+1) + '.txt'
    cam.append(lpt.math.Camera(file))
    intensity_max.append(255)
    useid_list.append(i)
cam_list = lpt.math.CamList(cam, intensity_max, useid_list)
print_dict(cam_list.to_dict())

# load 2d tracer 
tr2d_list_all = []
for i in range(4):
    file = '../../test/inputs/test_StereoMatch/pt2d_list_cam' + str(i+1) + '.csv'
    pt2d = np.loadtxt(file, delimiter=',')
    n_tr2d = pt2d.shape[0]
    print('cam'+str(i+1)+':', n_tr2d, 'tracers')
    
    tr2d = []
    for j in range(n_tr2d):
        tr2d.append(lpt.object.Tracer2D(lpt.math.Pt2D(pt2d[j,0], pt2d[j,1])))
        
    tr2d_list_all.append(tr2d)
    

cam_list: [<pyOpenLPT.math.Camera object at 0x000001D16308DE70>, <pyOpenLPT.math.Camera object at 0x000001D16308DDF0>, <pyOpenLPT.math.Camera object at 0x000001D16308DF30>, <pyOpenLPT.math.Camera object at 0x000001D16308DF70>]
intensity_max: [255, 255, 255, 255]
useid_list: [0, 1, 2, 3]
cam1: 32926 tracers
cam2: 32926 tracers
cam3: 32926 tracers
cam4: 32926 tracers


In [4]:
# initialize the stereomatch params 
param = lpt.stb.SMParam()
param.tor_2d = 1
param.tor_3d = 5e-3
param.n_thread = 6
param.check_id = 3
param.check_radius = 1
param.is_delete_ghost = False

print('StereoMatch Parameters:')
for key in param.to_dict().keys():
    print(key+':', param.to_dict()[key])
    

StereoMatch Parameters:
tor_2d: 1.0
tor_3d: 0.005
n_thread: 6
check_id: 3
check_radius: 1.0
is_delete_ghost: False
is_update_inner_var: False


In [6]:
# stereo match
sm = lpt.stb.StereoMatch(param, cam_list)
tr3d_list = sm.match(tr2d_list_all)

	Finish stereomatching: n_before_del = 34499.


In [7]:
pt3d = tr3d_list[0]._pt_center
pt3d.print(3)


Matrix = 
(dim_row, dim_col) = (3,1)
| -1.234e+01 |
| 4.884e+00 |
| -2.491e+00 |

